In [1]:
%pwd

'/Users/shashalin/Documents/Capstone/Viserion/CNN_LSTM'

In [2]:
aligned_RRM_path="../data/aligned_processed_RRM.csv"
from preprocessing import preprocess
vocab, df_aligned = preprocess(preprocessed=True, RRM_path=aligned_RRM_path,  
                               output_path='../data/aligned_processed_RRM.csv')
len(vocab)

Building vocabulary of amino acids...


26

In [55]:
from RRM_Sequence import RRM_Sequence, collate_fn
import pandas as pd
from train_decoder import to_var
from torch.utils.data import DataLoader
val_index = pd.read_csv('../data/val_index.csv',header=None).iloc[:,0]
val_loader = RRM_Sequence(df_aligned.loc[val_index, :], vocab)
val_loader = DataLoader(val_loader, batch_size=4, 
        shuffle=True, collate_fn=collate_fn)

In [82]:
from torch.nn.utils.rnn import pack_padded_sequence
def forward(decoder, features, captions, lengths):
        """Auto-encode RRM sequence vectors."""
        embeddings = decoder.embed(captions)
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True) 
        hiddens, (h_n, c_n) = decoder.lstm(packed)
        return h_n[0]
encoderCNN = ResNetEncoder(26, 128)
decoderRNN = DecoderRNN(128, 256, 26, 1)
for batch_idx, (names, rrms_aligned, rrms_unaligned, lengths) in enumerate(val_loader):
    rrms_aligned = to_var(rrms_aligned) 
    rrms_unaligned = to_var(rrms_unaligned)
    features = encoderCNN(rrms_aligned) 
    hiddens = forward(decoderRNN, features, rrms_unaligned, lengths)
    hiddens = hiddens.data.numpy()
    break

In [83]:
hiddens.shape

(4, 256)

In [88]:
df = pd.DataFrame(hiddens)
df['name'] = names
df.to_csv('hiddens.csv')

In [89]:
pd.read_csv('hiddens.csv', index_col=0)

,0,1,2,3,4,5,6,7,8,9,...,247,248,249,250,251,252,253,254,255,name
0,0.017652,0.041005,0.049488,0.016279,0.033113,-0.015965,-0.012665,-0.015118,-0.017834,-0.016842,...,0.011305,-0.001120,-0.034178,0.037863,-0.032957,-0.030762,-0.001791,0.025906,-0.035414,R1EZB0.1_1069-1138
1,0.017682,0.041622,0.053020,0.013998,0.033111,-0.017186,-0.014528,-0.012768,-0.017621,-0.012802,...,0.012216,0.000264,-0.034434,0.039452,-0.033339,-0.026146,0.000445,0.029595,-0.034215,S9WB43.1_63-132
2,0.017637,0.033685,0.041647,0.017008,0.034907,-0.021067,-0.010181,-0.013752,-0.013260,-0.014828,...,0.012627,-0.002972,-0.025138,0.052617,-0.033456,-0.026209,-0.003935,0.026459,-0.040626,G3VLX4.1_100-168
3,0.030574,0.038938,0.048841,0.005747,0.024591,-0.025730,-0.011422,-0.021432,-0.014457,-0.021913,...,0.004434,-0.018257,-0.023368,0.054208,-0.035712,-0.012639,-0.015041,0.040283,-0.036845,A0A091RIC4.1_213-277


In [94]:
import re
help(re.sub)

Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the match object and must return
    a replacement string to be used.



In [119]:
%rm ../data/tentative_genename_mapping.csv

In [120]:
import re
with open('../data/tentative_genename_mapping.tab', 'rb') as original_file:
    for i, line in enumerate(original_file):
        with open('../data/tentative_genename_mapping.csv', 'a') as new_file:
            line = re.sub(b'\t', b',', line)
            new_file.write(line.decode("utf-8") )

In [130]:
match = pd.read_csv('../data/tentative_genename_mapping.csv')
import numpy as np

(array(['1', '2', '3', '4', '5', '6', '7', '8', 'A', 'B', 'C', 'D', 'E',
        'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
        'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e',
        'f', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u',
        'v', 'x', 'y', 'z'],
       dtype='<U1'),
 array([1079,  304,   42,    9,  108,  143,    1,  198, 5184, 2321, 3665,
        2169, 2129,  947, 1983, 1804,  648,  464,  816, 2645, 3169, 2949,
        1218, 5646,  135, 2496, 3015, 3207, 1003, 1319,  604,  376,  157,
         773,   36,   46,  143,   30,  126,   22,   25,   84,   11,    6,
          49,   51,    8,  111,  310,  310,   99,   36,  150,    1,    3,
          24]))

In [180]:
beginning, counts = np.unique([to.split('_')[0][:3] for to in match['To']], return_counts=True)
name_count_pairs = sorted(zip(beginning, counts), key=lambda x: x[1], reverse=True)

In [182]:
genes_to_plot = [name for name, count in name_count_pairs[:10]]

In [125]:
len(np.unique([to.split('_')[0] for to in match['To']]))

9662

In [17]:
representations = []
for i, (name, RRM_aligned, RRM_unaligned) in enumerate(val_loader):
    RRM_aligned = to_var(RRM_aligned)
    RRM_unaligned = to_var(RRM_unaligned)
    features = encoder(rrms_aligned) 
    outputs = decoder(features, rrms_unaligned, lengths)
    

8966

In [14]:
def collate_fn(data):
    data.sort(key=lambda x: len(x[2]), reverse=True)
    names, rrms_aligned, rrms_unaligned = zip(*data)
    rrms_aligned = torch.stack(rrms_aligned, 0).long()

    lengths = [len(rrm) for rrm in rrms_unaligned]
    unaligned = torch.zeros(len(rrms_unaligned), max(lengths)).long()
    for i, rrm in enumerate(rrms_unaligned):
        end = lengths[i]
        unaligned[i, :end] = rrm[:end]
    return names, rrms_aligned, unaligned, lengths

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
C5YAM7.1_625-686,<start>,L,Y,V,T,N,L,-,P,Q,...,Y,L,I,E,G,K,R,L,-,<end>
A0A176VP33.1_152-222,<start>,-,F,V,K,G,F,-,D,K,...,Q,E,V,N,G,Q,T,L,-,<end>
A0A0D2X2J0.1_428-495,<start>,L,C,I,R,N,L,-,G,E,...,K,P,S,P,G,T,Q,-,-,<end>
A0A0D2X2J0.1_797-859,<start>,L,Y,L,T,N,V,-,D,F,...,Q,I,-,-,-,-,-,-,-,<end>
A0A0E0PFX5.1_389-458,<start>,L,Y,V,C,N,L,-,S,L,...,R,L,V,D,G,R,K,I,-,<end>
A0A0E0PFX5.1_598-659,<start>,L,Y,I,T,N,L,-,P,R,...,Y,M,V,E,G,K,R,L,-,<end>
M7ZQI3.1_633-694,<start>,L,Y,V,G,G,F,-,P,P,...,Y,Q,I,G,A,S,T,L,-,<end>
W5AYN4.1_442-505,<start>,L,Y,V,C,N,I,-,P,A,...,S,L,V,E,G,E,T,L,-,<end>
A0A194QSP4.1_352-421,<start>,I,F,V,R,N,L,-,P,Y,...,T,S,F,C,G,R,M,L,-,<end>
A0A194QSP4.1_512-581,<start>,I,F,V,R,N,L,-,P,Y,...,T,S,F,C,G,R,M,L,-,<end>


In [10]:
import sys
import os
sys.path.append(os.getcwd()+'/'+'../util')
import tsne
help(tsne)

Help on module tsne:

NAME
    tsne

FUNCTIONS
    tsne_plot(tsne_plot_name, labels, latent_representation, take_first_n=500, perplexity=40, n_iter=1500, n_components=2)
        list int, str labels: color point according to this color 
        np.ndarray latent_representation: [ Num of sequences X Dim of Latent Rep. ]
        take_first_n: only first n row will show in tsne projection

FILE
    /Users/shashalin/Documents/Capstone/Viserion/util/tsne.py




In [151]:
hiddens=pd.read_csv('hiddens.csv', index_col=0)

In [159]:
hiddens.shape

(8966, 257)

In [167]:
hiddens['name1'] = [name.split('.')[0] for name in hiddens.name]

In [177]:
matched = pd.merge(hiddens, match, left_on='name1', right_on='From')

In [183]:
matched['genID'] = [to[:3] for to in matched['To']]

In [187]:
[gene in genes_to_plot for gene in matched['genID']]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 Fals

In [212]:
# hiddens_to_plot = matched.loc[[gene in genes_to_plot for gene in matched['genID']], :].drop(
#     ['name','name1', 'From', 'To'], 1)
hiddens_to_plot = matched.drop(
    ['name','name1', 'From', 'To'], 1)

In [216]:
hiddens_to_plot['genID'].tolist()

['LOT',
 'PB0',
 'VHE',
 'L48',
 'PLE',
 'N82',
 'WN5',
 'DAC',
 'ALC',
 'D62',
 'K44',
 'PPA',
 'G21',
 'POV',
 'DOT',
 'FG0',
 'VIT',
 'GSO',
 'K43',
 'SOV',
 'RBM',
 'K44',
 'POL',
 'ST4',
 'Mal',
 'I79',
 'Cto',
 'RBP',
 'RBP',
 'T55',
 'T55',
 'HMP',
 'FIB',
 'A1C',
 'A1C',
 'SNR',
 'hnr',
 'CLF',
 'RCO',
 'L79',
 'APZ',
 'TIA',
 'AM5',
 'VSA',
 'W97',
 'GW7',
 'OT_',
 'Smp',
 'Dme',
 'ASU',
 'CAO',
 'tnr',
 'KK1',
 'PmU',
 'H63',
 'MS3',
 'N33',
 'ACY',
 'K50',
 'UCR',
 'M40',
 'A1O',
 'LOC',
 'LOC',
 'AK8',
 'FOM',
 'RAL',
 'L79',
 'PHA',
 'H07',
 'AV2',
 'B45',
 'WN5',
 'AN9',
 'BN1',
 'TCM',
 'L22',
 'AND',
 'DIC',
 'DIC',
 '102',
 'CGG',
 'A1O',
 'JCG',
 'JCG',
 'SOR',
 'AK8',
 'COC',
 'VHE',
 'RFI',
 'Dpe',
 'AMA',
 'HNR',
 'UCR',
 'LOC',
 'LOC',
 'PHY',
 'BRA',
 'Spx',
 'ANO',
 'Dbu',
 'MDA',
 'SOR',
 'BN1',
 'Vbr',
 'Bna',
 'PST',
 'PV1',
 'N32',
 'YME',
 'G5I',
 'N32',
 'PTS',
 'PCO',
 'CEL',
 'I79',
 'A1O',
 'A1O',
 'GLY',
 'UCR',
 'MUC',
 'ACL',
 'AV2',
 'AV2',
 'TRE',


In [226]:
from imp import reload
reload(tsne)
tsne.tsne_plot('Representation_from_CNN_LSTM', hiddens_to_plot['genID'].tolist(), 
               hiddens_to_plot.drop(['genID'], 1).values, perplexity=100)

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 301 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 500 / 500
[t-SNE] Mean sigma: 3.138235
[t-SNE] KL divergence after 100 iterations with early exaggeration: 0.281688
[t-SNE] Error after 175 iterations: 0.281688
(500, 2) 274


TypeError: unhashable type: 'list'

In [220]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np 
labels=hiddens_to_plot['genID'].tolist()
unique_labels = list(set(labels))
colors = [ unique_labels.index(l) for l in labels]
colors = iter(cm.rainbow(np.linspace(0, 1, len(unique_labels))))

In [225]:
next(colors).tolist()

[0.5, 0.0, 1.0, 1.0]

In [223]:
len(unique_labels)

959